### 模型定义

In [1]:
import torch
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.convd1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.convd2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.convd3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.5)

    
    def forward(self, x):
        x = self.convd1(x)
        x = self.pool(self.relu(x))
        x = self.convd2(x)
        x = self.pool(self.relu(x))
        x = self.convd3(x)
        x = self.pool(self.relu(x))
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

### 准备数据 & 配置日志功能

In [2]:
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import logging
import sys
import os
from tqdm.notebook import tqdm
# ==========================================
# 1. 配置日志 (保持不变)
# ==========================================
def get_logger(filename='training.log'):
    logger = logging.getLogger('cifar_trainer')
    logger.setLevel(logging.INFO)
    if logger.hasHandlers(): return logger
    formatter = logging.Formatter('%(asctime)s | %(message)s', datefmt='%H:%M:%S')
    file_handler = logging.FileHandler(filename, mode='w')
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)
    return logger

logger = get_logger()

# ==========================================
# 2. 数据准备 (保持不变)
# ==========================================
input_source = '/kaggle/input/cifar10-python/cifar-10-batches-py'
# 期望数据存放的位置
working_dest = './data/cifar-10-batches-py'
# 搬运数据
if not os.path.exists(working_dest):
    print(f"正在将数据从 {input_source} 复制到 {working_dest} ...")
    # 创建父目录 ./data
    os.makedirs(os.path.dirname(working_dest), exist_ok=True)
    # 复制整个文件夹
    try:
        shutil.copytree(input_source, working_dest)
        print("数据复制完成！")
    except FileNotFoundError:
        print("❌ 错误：找不到源路径，请检查 input_source 是否正确！")
        # 打印一下 input 目录里到底有什么，方便调试
        print("Input 目录下的文件:", os.listdir('/kaggle/input'))
else:
    print("数据已在 Working 目录，跳过复制。")

def get_data_loaders(batch_size=128):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])
    train_dataset = datasets.CIFAR10(root='./data', train=True, download=False, transform=transform)
    test_dataset = datasets.CIFAR10(root='./data', train=False, download=False, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    return train_loader, test_loader

正在将数据从 /kaggle/input/cifar10-python/cifar-10-batches-py 复制到 ./data/cifar-10-batches-py ...
数据复制完成！


### 训练逻辑

In [3]:
# ==========================================
# 4. 训练逻辑 (这里修改了！增加了实时准确率)
# ==========================================
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    
    # 定义进度条
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}", leave=False)
    
    running_loss = 0.0
    correct_samples = 0
    total_samples = 0
    
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        # --- 计算实时指标 ---
        running_loss += loss.item()
        
        # 获取预测结果
        pred = output.argmax(dim=1, keepdim=True)
        # 累计预测正确的数量
        correct_samples += pred.eq(target.view_as(pred)).sum().item()
        # 累计总样本数
        total_samples += target.size(0)
        
        # 计算当前累积准确率
        current_acc = 100. * correct_samples / total_samples
        current_avg_loss = running_loss / (batch_idx + 1)
        
        # --- 更新进度条后缀 ---
        # 现在的显示格式：Loss: 1.2345 | Acc: 45.67%
        pbar.set_postfix({
            'loss': f'{current_avg_loss:.4f}', 
            'acc': f'{current_acc:.2f}%'
        })

    return current_avg_loss, current_acc

def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)
    acc = 100. * correct / len(test_loader.dataset)
    return test_loss, acc

### 超参数定义 & 开始训练

In [4]:
# ==========================================
# 5. 主程序运行
# ==========================================
BATCH_SIZE = 2048
EPOCHS = 50
LEARNING_RATE = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

logger.info(f"Using device: {DEVICE}")

train_loader, test_loader = get_data_loaders(BATCH_SIZE)
model = CNN().to(DEVICE)
if torch.cuda.device_count() > 1:
    print(f"检测到 {torch.cuda.device_count()} 块 GPU，已开启 DataParallel 并行加速！")
    # 这行代码会自动把任务分配给两块卡
    model = nn.DataParallel(model)
else:
    print("使用单卡训练")
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

logger.info("Starting Training...")
best_acc = 0.0
for epoch in range(1, EPOCHS + 1):
    train_loss, train_acc = train(model, DEVICE, train_loader, optimizer, criterion, epoch)
    val_loss, val_acc = test(model, DEVICE, test_loader, criterion)
    current_lr = optimizer.param_groups[0]['lr']
    logger.info(f"Epoch: {epoch} | LR: {current_lr:.6f}| Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pt')
        print(f"🔥 恭喜! 发现新高分: {best_acc:.2f}%, 模型已保存!")

logger.info("Training Finished.")

08:35:35 | Using device: cuda
检测到 2 块 GPU，已开启 DataParallel 并行加速！
08:35:36 | Starting Training...


Epoch 1:   0%|          | 0/25 [00:00<?, ?it/s]

08:35:46 | Epoch: 1 | LR: 0.001000| Train Loss: 1.9562 | Train Acc: 28.60% | Val Loss: 1.6900 | Val Acc: 39.43%
🔥 恭喜! 发现新高分: 39.43%, 模型已保存!


Epoch 2:   0%|          | 0/25 [00:00<?, ?it/s]

08:35:54 | Epoch: 2 | LR: 0.001000| Train Loss: 1.5788 | Train Acc: 42.57% | Val Loss: 1.4198 | Val Acc: 48.79%
🔥 恭喜! 发现新高分: 48.79%, 模型已保存!


...

08:41:53 | Epoch: 48 | LR: 0.001000| Train Loss: 0.3496 | Train Acc: 87.60% | Val Loss: 0.7662 | Val Acc: 75.61%


Epoch 49:   0%|          | 0/25 [00:00<?, ?it/s]

08:42:01 | Epoch: 49 | LR: 0.001000| Train Loss: 0.3470 | Train Acc: 87.68% | Val Loss: 0.7522 | Val Acc: 75.65%


Epoch 50:   0%|          | 0/25 [00:00<?, ?it/s]

08:42:09 | Epoch: 50 | LR: 0.001000| Train Loss: 0.3305 | Train Acc: 88.45% | Val Loss: 0.7413 | Val Acc: 76.16%
🔥 恭喜! 发现新高分: 76.16%, 模型已保存!
08:42:09 | Training Finished.
